#HW5
##Esteban Velasquez
##114502904
##02/10/2025

In [ ]:
!pip install --quiet huggingface_hub pandas
from huggingface_hub import InferenceClient
import pandas as pd
import numpy as np
from google.colab import drive

# --------------------------
# Step 1: Setup HF API
# --------------------------
HF_API_KEY = "key"  # 👈 paste your key here
client = InferenceClient(model="mistralai/Mistral-7B-Instruct-v0.2", token=HF_API_KEY)

# --------------------------
# Step 2: Load Dataset
# --------------------------
drive.mount('/content/drive')
file_path = "/content/drive/My Drive/googleplaystore.csv"
df = pd.read_csv(file_path)

# --------------------------
# Step 3: Clean Data
# --------------------------
df['Installs'] = df['Installs'].astype(str).str.replace('[+,]', '', regex=True)
df['Installs'] = pd.to_numeric(df['Installs'], errors='coerce')
df['Reviews'] = pd.to_numeric(df['Reviews'], errors='coerce')

if 'Price' in df.columns:
    df['Price'] = df['Price'].astype(str).str.replace('$', '', regex=False)
    df['Price'] = pd.to_numeric(df['Price'], errors='coerce')

def size_to_num(x):
    if isinstance(x, str):
        x = x.strip()
        if x.endswith('M'):
            return float(x[:-1]) * 1e6
        elif x.endswith('k'):
            return float(x[:-1]) * 1e3
        elif x == 'Varies with device':
            return np.nan
    return np.nan

if 'Size' in df.columns:
    df['Size'] = df['Size'].apply(size_to_num)

# --------------------------
# Step 4: Summarize Dataset
# --------------------------
num_vars = df.select_dtypes(include=['int64','float64']).columns
cat_vars = df.select_dtypes(include=['object']).columns

summary_text = f"""
Google Play Store dataset:
- Shape: {df.shape}
- Numerical variables: {list(num_vars)}
- Categorical variables: {list(cat_vars)}
- Numerical summary:
{df[num_vars].describe().to_string()}
- Missing values:
{df[num_vars].isnull().sum().to_dict()}
- Categorical counts (top 3 per column):
{ {col: df[col].value_counts().head(3).to_dict() for col in cat_vars} }
- Top Genres per Category (sample):
{df.groupby("Category")["Genres"].value_counts().head(10).to_dict()}
- Correlation Installs vs Reviews:
{df['Installs'].corr(df['Reviews'])}
- Top 3 Categories:
{df['Category'].value_counts().head(3).to_dict()}
- Top 3 Highest Rated Apps:
{df.sort_values(by='Rating', ascending=False).head(3)[['App','Rating','Installs']].to_dict(orient='records')}
"""

# --------------------------
# Step 5: Send to HF Chat LLM
# --------------------------
prompt = f"""
Here is a dataset summary:

{summary_text}

Please answer in clear English:
1. Overview of the dataset
2. How many numerical vs categorical variables
3. Summary statistics for numerical variables (mean, std, min, max, missing)
4. Category counts for categorical variables
5. Relationship between Genres and Category
6. Relationship between Installs and Reviews
7. Top 3 most popular Categories (popularity = # of apps)
8. Top 3 apps with highest ratings — does install count matter?
"""

response = client.chat.completions.create(
    model="mistralai/Mistral-7B-Instruct-v0.2",
    messages=[{"role": "user", "content": prompt}],
    max_tokens=800,
)

print(response.choices[0].message["content"])


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


ValueError: Model mistralai/Mistral-7B-Instruct-v0.2 is not supported for task text-generation and provider featherless-ai. Supported task: conversational.